In [1]:
import matplotlib.pyplot as plt
import numpy as np
import csv

%matplotlib inline

In [2]:
f = open('../data/test.csv', 'r')
csv_reader = csv.DictReader(f)

In [3]:
rows = [row for row in csv_reader]
sequences = {int(row['Id']): [int(s) for s in row['Sequence'].split(',')] for row in rows}
print(len(sequences.keys()))

113845


# Monotonicity

In [4]:
def strictly_increasing(A):
    return all(x < y for x, y in zip(A, A[1:]))
    
def non_decreasing(A):
    return all(x <= y for x, y in zip(A, A[1:]))
    
def strictly_decreasing(A):
    return all(x > y for x, y in zip(A, A[1:]))
    
def non_increasing(A):
    return all(x >= y for x, y in zip(A, A[1:]))

def non_monotonic(A):
    return not any([strictly_increasing(A), strictly_decreasing(A), non_increasing(A), non_decreasing(A)])

In [5]:
A0 = list(filter(strictly_increasing, sequences.values()))
A1 = list(filter(non_decreasing, sequences.values()))
A2 = list(filter(strictly_decreasing, sequences.values()))
A3 = list(filter(non_increasing, sequences.values()))
A4 = list(filter(non_monotonic, sequences.values()))

As = [A0,A1,A2,A3,A4]
Ns = ["strictly increasing", "non-decreasing", "strictly increasing", "non-increasing", "non-monotonic"]

print(len(A0))
print(len(A1))
print(len(A2))
print(len(A3))
print(len(A4))

60002
72585
272
468
40870


# most likely next number

In [6]:
mode = lambda a: max(map(lambda val: (a.count(val), val), set(a)))[1]

In [7]:
from itertools import tee

def nwise(iterable, n):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    iterables = tee(iterable, n)
    for i, it in enumerate(iterables):
        for j in range(i):
            next(it,None)
    return zip(*iterables)

In [8]:
from collections import Counter

def counts(As, n=1, end=None, normalized = False):
    As = list(map(lambda A: nwise(A,n), As))
    elems = [e for A in As for e in A[-end:]] if end else [e for A in As for e in A]
    hist = Counter(elems)
    if normalized:
        N=sum(hist.values())
        return {e: cnt/N for e, cnt in hist.items()}
    return hist

In [9]:
from collections import defaultdict

def nwise_dict(As,n=1, prior=False):
    cnts = counts(As, n=n+1, normalized=True)
    X = list(zip(*cnts.keys()))
    keys = list(zip(*X[:-1]))
    values = X[-1]
    
    weights = cnts.values()
    wprior = counts(As, normalized=True)

    d = defaultdict(lambda : defaultdict(int))
    for key, value, w in zip(keys, values, weights):
        d[key][value] = w * (wprior[value,] if prior else 1)
    return d

In [10]:
def max_next(nwise_d):
    return {key: max(value.items(), key=lambda x: x[1]) for key, value in nwise_d.items()}

In [11]:
def predict(As, threshold, n=1, prior=False):
    
    d = max_next(nwise_dict(As, n=n, prior=prior))
    
    def pred(A):
        
        return d[tuple(A[-n:])][0] if tuple(A[-n:]) in d and d[tuple(A[-n:])][1] >= threshold else None
    return pred

# Testing

In [12]:
def predict_test(As, threshold, n=1, prior=False, fallback=False):
    As = list(map(lambda A: A[:-1], As))
    d = max_next(nwise_dict(As, n=n, prior=prior))
    
    def pred(A):
        fb = mode(A) if fallback and len(A) > 0 else None
        return d[tuple(A[-n:])][0] if tuple(A[-n:]) in d and d[tuple(A[-n:])][1] >= threshold else fb
    return pred

In [13]:
def test(As, predict):
    expected = [A[-1] for A in As]
    predicted = [predict(A[:-1]) for A in As]
    
    total_right = len([1 for e, p in zip(expected, predicted) if e == p])
    accuracy = total_right/len(As)
    precision = total_right/max(len([1 for p in predicted if p is not None]),1)
    return total_right, accuracy, precision

# Fit

In [25]:
p = predict_test(list(sequences.values()), 1.0e-4)
for A, N in zip(As, Ns):
    total_right, accuracy, precision = test(A,p)
    print(N, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

strictly increasing :  accuracy =  0.021499283357221427 precision =  0.2723242558581381 total =  1290
non-decreasing :  accuracy =  0.02444031135909623 precision =  0.27406148617333537 total =  1774
strictly increasing :  accuracy =  0.0 precision =  0.0 total =  0
non-increasing :  accuracy =  0.21367521367521367 precision =  0.45454545454545453 total =  100
non-monotonic :  accuracy =  0.10589674577930022 precision =  0.18861675237514164 total =  4328


# Threshold

In [36]:
for thresh in [1.0e-1, 1.0e-2,1.0e-3,1.0e-4,1.0e-5]:
    total_right, accuracy, precision = test(A3, predict_test(A3, thresh))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.1 :  accuracy =  0.19230769230769232 precision =  0.9375 total =  90
0.01 :  accuracy =  0.2692307692307692 precision =  0.9264705882352942 total =  126
0.001 :  accuracy =  0.3141025641025641 precision =  0.8258426966292135 total =  147
0.0001 :  accuracy =  0.358974358974359 precision =  0.7850467289719626 total =  168
1e-05 :  accuracy =  0.37606837606837606 precision =  0.6641509433962264 total =  176
1e-06 :  accuracy =  0.37606837606837606 precision =  0.6641509433962264 total =  176
1e-07 :  accuracy =  0.37606837606837606 precision =  0.6641509433962264 total =  176
1e-08 :  accuracy =  0.37606837606837606 precision =  0.6641509433962264 total =  176


In [57]:
for thresh in [1.0e-1, 1.0e-2,1.0e-3,1.0e-4,1.0e-5]:
    total_right, accuracy, precision = test(A2, predict_test(A2, thresh))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.1 :  accuracy =  0.0 precision =  0.0 total =  0
0.01 :  accuracy =  0.0 precision =  0.0 total =  0
0.001 :  accuracy =  0.07352941176470588 precision =  0.5263157894736842 total =  20
0.0001 :  accuracy =  0.11029411764705882 precision =  0.2912621359223301 total =  30
1e-05 :  accuracy =  0.11029411764705882 precision =  0.2912621359223301 total =  30


In [38]:
for thresh in [1.0e-1, 1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A1, predict_test(A1, thresh))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.1 :  accuracy =  0.0 precision =  0.0 total =  0
0.01 :  accuracy =  0.0 precision =  0.0 total =  0
0.001 :  accuracy =  0.001735895846249225 precision =  0.10465116279069768 total =  126
0.0001 :  accuracy =  0.032458496934628364 precision =  0.2860264659463397 total =  2356
1e-05 :  accuracy =  0.04619411724185438 precision =  0.2253057384760113 total =  3353
1e-06 :  accuracy =  0.05607219122408211 precision =  0.16184189597582313 total =  4070
1e-07 :  accuracy =  0.06849900117104085 precision =  0.1464377227343681 total =  4972
1e-08 :  accuracy =  0.06849900117104085 precision =  0.1464377227343681 total =  4972


In [44]:
for thresh in [1.0e-1, 1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A0, predict_test(A0, thresh))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.1 :  accuracy =  0.0 precision =  0.0 total =  0
0.01 :  accuracy =  0.0 precision =  0.0 total =  0
0.001 :  accuracy =  0.0007333088897036765 precision =  0.08193668528864059 total =  44
0.0001 :  accuracy =  0.03491550281657278 precision =  0.3075455079271873 total =  2095
1e-05 :  accuracy =  0.049581680610646314 precision =  0.22939316832446605 total =  2975
1e-06 :  accuracy =  0.058981367287757075 precision =  0.16437529029261497 total =  3539
1e-07 :  accuracy =  0.0724309189693677 precision =  0.14973814773980154 total =  4346
1e-08 :  accuracy =  0.0724309189693677 precision =  0.14973814773980154 total =  4346


In [51]:
for thresh in [1.0e-1, 1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A4, predict_test(A4, thresh))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.1 :  accuracy =  0.0 precision =  0.0 total =  0
0.01 :  accuracy =  0.06753119647663322 precision =  0.3553038105046344 total =  2760
0.001 :  accuracy =  0.08977244922926352 precision =  0.24772128823171966 total =  3669
0.0001 :  accuracy =  0.10024467824810375 precision =  0.20251099797340716 total =  4097
1e-05 :  accuracy =  0.10660631269880108 precision =  0.18418938913548932 total =  4357
1e-06 :  accuracy =  0.11096158551504771 precision =  0.162225004471472 total =  4535
1e-07 :  accuracy =  0.1148764374847076 precision =  0.14519870109788155 total =  4695
1e-08 :  accuracy =  0.1148764374847076 precision =  0.14519870109788155 total =  4695


# Fallback

In [52]:
for fallback in [True, False]:
    total_right, accuracy, precision = test(A3, predict_test(A3, .1, fallback=fallback))
    print("Fallback = ", fallback, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

Fallback =  True :  accuracy =  0.23931623931623933 precision =  0.27184466019417475 total =  112
Fallback =  False :  accuracy =  0.19230769230769232 precision =  0.9375 total =  90


In [60]:
for fallback in [True, False]:
    total_right, accuracy, precision = test(A2, predict_test(A2, .00001, fallback=fallback))
    print("Fallback = ", fallback, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

Fallback =  True :  accuracy =  0.11029411764705882 precision =  0.1388888888888889 total =  30
Fallback =  False :  accuracy =  0.11029411764705882 precision =  0.2912621359223301 total =  30


## non-decreasing

In [53]:
for fallback in [True, False]:
    total_right, accuracy, precision = test(A1, predict_test(A1, .0001, fallback=fallback))
    print("Fallback = ", fallback, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

Fallback =  True :  accuracy =  0.033409106564717225 precision =  0.03343490190130844 total =  2425
Fallback =  False :  accuracy =  0.032458496934628364 precision =  0.2860264659463397 total =  2356


In [65]:
for fallback in [True, False]:
    total_right, accuracy, precision = test(A1, predict_test(A1, .00001, fallback=fallback))
    print("Fallback = ", fallback, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

Fallback =  True :  accuracy =  0.04681408004408624 precision =  0.04685022542707055 total =  3398
Fallback =  False :  accuracy =  0.04619411724185438 precision =  0.2253057384760113 total =  3353


## increasing

In [54]:
for fallback in [True, False]:
    total_right, accuracy, precision = test(A0, predict_test(A0, .0001, fallback=fallback))
    print("Fallback = ", fallback, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

Fallback =  True :  accuracy =  0.03491550281657278 precision =  0.034948119974643845 total =  2095
Fallback =  False :  accuracy =  0.03491550281657278 precision =  0.3075455079271873 total =  2095


In [62]:
for fallback in [True, False]:
    total_right, accuracy, precision = test(A0, predict_test(A0, .00001, fallback=fallback))
    print("Fallback = ", fallback, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

Fallback =  True :  accuracy =  0.049581680610646314 precision =  0.04962799853201214 total =  2975
Fallback =  False :  accuracy =  0.049581680610646314 precision =  0.22939316832446605 total =  2975


In [64]:
for fallback in [True, False]:
    total_right, accuracy, precision = test(A0, predict_test(A0, .000001, fallback=fallback))
    print("Fallback = ", fallback, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

Fallback =  True :  accuracy =  0.058981367287757075 precision =  0.05903646615287092 total =  3539
Fallback =  False :  accuracy =  0.058981367287757075 precision =  0.16437529029261497 total =  3539


## non-mono

In [55]:
for fallback in [True, False]:
    total_right, accuracy, precision = test(A4, predict_test(A4, .01, fallback=fallback))
    print("Fallback = ", fallback, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

Fallback =  True :  accuracy =  0.1345241008074382 precision =  0.1345241008074382 total =  5498
Fallback =  False :  accuracy =  0.06753119647663322 precision =  0.3553038105046344 total =  2760


In [63]:
for fallback in [True, False]:
    total_right, accuracy, precision = test(A4, predict_test(A4, .001, fallback=fallback))
    print("Fallback = ", fallback, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

Fallback =  True :  accuracy =  0.12304869097137265 precision =  0.12304869097137265 total =  5029
Fallback =  False :  accuracy =  0.08977244922926352 precision =  0.24772128823171966 total =  3669


# Num Back

In [22]:
for A, N in zip(As, Ns):
    total_right, accuracy, precision = test(A, predict_test(A, 1.0e-4))
    print(N, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

strictly increasing :  accuracy =  0.03491550281657278 precision =  0.3075455079271873 total =  2095
non-decreasing :  accuracy =  0.032458496934628364 precision =  0.2860264659463397 total =  2356
strictly increasing :  accuracy =  0.11029411764705882 precision =  0.2912621359223301 total =  30
non-increasing :  accuracy =  0.358974358974359 precision =  0.7850467289719626 total =  168
non-monotonic :  accuracy =  0.10024467824810375 precision =  0.20251099797340716 total =  4097


In [23]:
for A, N in zip(As, Ns):
    total_right, accuracy, precision = test(A, predict_test(A, 1.0e-6, n=2))
    print(N, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

strictly increasing :  accuracy =  0.06346455118162728 precision =  0.32205683355886333 total =  3808
non-decreasing :  accuracy =  0.06522008679479231 precision =  0.3331691181645436 total =  4734
strictly increasing :  accuracy =  0.08455882352941177 precision =  0.5476190476190477 total =  23
non-increasing :  accuracy =  0.3611111111111111 precision =  0.8407960199004975 total =  169
non-monotonic :  accuracy =  0.1413261561047223 precision =  0.2862523540489642 total =  5776


# Prior

In [26]:
for A, N in zip(As, Ns):
    total_right, accuracy, precision = test(A, predict_test(A, 1.0e-4, prior=True))
    print(N, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

strictly increasing :  accuracy =  0.0 precision =  0.0 total =  0
non-decreasing :  accuracy =  0.00012399256044637322 precision =  0.08823529411764706 total =  9
strictly increasing :  accuracy =  0.0 precision =  0.0 total =  0
non-increasing :  accuracy =  0.2692307692307692 precision =  0.9264705882352942 total =  126
non-monotonic :  accuracy =  0.08690971372644972 precision =  0.2398217541016812 total =  3552


# Numback with thresh

In [67]:
for thresh in [1.0e-1, 1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A4, predict_test(A4,thresh,n=1))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.1 :  accuracy =  0.0 precision =  0.0 total =  0
0.01 :  accuracy =  0.03748470760949352 precision =  0.644509886411443 total =  1532
0.001 :  accuracy =  0.07200880841693173 precision =  0.42430795847750863 total =  2943
0.0001 :  accuracy =  0.10325422069977978 precision =  0.3176993149137996 total =  4220
1e-05 :  accuracy =  0.12417421091264987 precision =  0.3079116612061643 total =  5075
1e-06 :  accuracy =  0.1413261561047223 precision =  0.2862523540489642 total =  5776
1e-07 :  accuracy =  0.15632493271348177 precision =  0.2600643139170432 total =  6389
1e-08 :  accuracy =  0.15632493271348177 precision =  0.2600643139170432 total =  6389
1e-09 :  accuracy =  0.15632493271348177 precision =  0.2600643139170432 total =  6389


In [67]:
for thresh in [1.0e-1, 1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A4, predict_test(A4,thresh,n=2))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.1 :  accuracy =  0.0 precision =  0.0 total =  0
0.01 :  accuracy =  0.03748470760949352 precision =  0.644509886411443 total =  1532
0.001 :  accuracy =  0.07200880841693173 precision =  0.42430795847750863 total =  2943
0.0001 :  accuracy =  0.10325422069977978 precision =  0.3176993149137996 total =  4220
1e-05 :  accuracy =  0.12417421091264987 precision =  0.3079116612061643 total =  5075
1e-06 :  accuracy =  0.1413261561047223 precision =  0.2862523540489642 total =  5776
1e-07 :  accuracy =  0.15632493271348177 precision =  0.2600643139170432 total =  6389
1e-08 :  accuracy =  0.15632493271348177 precision =  0.2600643139170432 total =  6389
1e-09 :  accuracy =  0.15632493271348177 precision =  0.2600643139170432 total =  6389


In [68]:
for thresh in [1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A4, predict_test(A4,thresh,n=3))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.01 :  accuracy =  0.028358208955223882 precision =  0.7614980289093298 total =  1159
0.001 :  accuracy =  0.049253731343283584 precision =  0.621104597346498 total =  2013
0.0001 :  accuracy =  0.08128211402006362 precision =  0.48376292412989663 total =  3322
1e-05 :  accuracy =  0.11702960606802056 precision =  0.36747080516287645 total =  4783
1e-06 :  accuracy =  0.1424516760459995 precision =  0.36943968525921694 total =  5822
1e-07 :  accuracy =  0.16501101052116468 precision =  0.35037406483790523 total =  6744
1e-08 :  accuracy =  0.16501101052116468 precision =  0.35037406483790523 total =  6744
1e-09 :  accuracy =  0.16501101052116468 precision =  0.35037406483790523 total =  6744


In [69]:
for thresh in [1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A4, predict_test(A4,thresh,n=4))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.01 :  accuracy =  0.01668705652067531 precision =  0.8276699029126213 total =  682
0.001 :  accuracy =  0.03924639099584047 precision =  0.7016622922134733 total =  1604
0.0001 :  accuracy =  0.06290677758747247 precision =  0.6183261183261183 total =  2571
1e-05 :  accuracy =  0.09674577930022021 precision =  0.5372282608695652 total =  3954
1e-06 :  accuracy =  0.14061658918522144 precision =  0.426461858118136 total =  5747
1e-07 :  accuracy =  0.16848544164423782 precision =  0.42778157420637386 total =  6886
1e-08 :  accuracy =  0.16848544164423782 precision =  0.42778157420637386 total =  6886
1e-09 :  accuracy =  0.16848544164423782 precision =  0.42778157420637386 total =  6886


# fallback and nums

In [70]:
for thresh in [1.0e-1, 1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A4, predict_test(A4,thresh,n=1,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.1 :  accuracy =  0.14306337166625888 precision =  0.14306337166625888 total =  5847
0.01 :  accuracy =  0.1345241008074382 precision =  0.1345241008074382 total =  5498
0.001 :  accuracy =  0.12304869097137265 precision =  0.12304869097137265 total =  5029
0.0001 :  accuracy =  0.11918277465133349 precision =  0.11918277465133349 total =  4871
1e-05 :  accuracy =  0.12091999021287007 precision =  0.12091999021287007 total =  4942
1e-06 :  accuracy =  0.11989234157083435 precision =  0.11989234157083435 total =  4900
1e-07 :  accuracy =  0.11930511377538537 precision =  0.11930511377538537 total =  4876
1e-08 :  accuracy =  0.11930511377538537 precision =  0.11930511377538537 total =  4876


In [71]:
for thresh in [1.0e-1, 1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A4, predict_test(A4,thresh,n=2,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.1 :  accuracy =  0.14306337166625888 precision =  0.14306337166625888 total =  5847
0.01 :  accuracy =  0.14377293858575974 precision =  0.14377293858575974 total =  5876
0.001 :  accuracy =  0.14117934915586006 precision =  0.14117934915586006 total =  5770
0.0001 :  accuracy =  0.14519207242476143 precision =  0.14519207242476143 total =  5934
1e-05 :  accuracy =  0.1540004893564962 precision =  0.1540004893564962 total =  6294
1e-06 :  accuracy =  0.16256422804012724 precision =  0.16256422804012724 total =  6644
1e-07 :  accuracy =  0.17066307805236114 precision =  0.17066307805236114 total =  6975
1e-08 :  accuracy =  0.17066307805236114 precision =  0.17066307805236114 total =  6975


In [72]:
for thresh in [1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A4, predict_test(A4,thresh,n=3,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.01 :  accuracy =  0.14396868118424272 precision =  0.14396868118424272 total =  5884
0.001 :  accuracy =  0.14470271592855394 precision =  0.14470271592855394 total =  5914
0.0001 :  accuracy =  0.14949840959138733 precision =  0.14949840959138733 total =  6110
1e-05 :  accuracy =  0.16187912894543674 precision =  0.16187912894543674 total =  6616
1e-06 :  accuracy =  0.17648642035723025 precision =  0.17648642035723025 total =  7213
1e-07 :  accuracy =  0.19021287007585025 precision =  0.19021287007585025 total =  7774
1e-08 :  accuracy =  0.19021287007585025 precision =  0.19021287007585025 total =  7774


In [73]:
for thresh in [1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A4, predict_test(A4,thresh,n=4,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.01 :  accuracy =  0.14343038903841449 precision =  0.14343038903841449 total =  5862
0.001 :  accuracy =  0.1440910203082946 precision =  0.1440910203082946 total =  5889
0.0001 :  accuracy =  0.14864203572302423 precision =  0.14864203572302423 total =  6075
1e-05 :  accuracy =  0.16315145583557622 precision =  0.16315145583557622 total =  6668
1e-06 :  accuracy =  0.18333741130413506 precision =  0.18333741130413506 total =  7493
1e-07 :  accuracy =  0.20203082945926107 precision =  0.20203082945926107 total =  8257
1e-08 :  accuracy =  0.20203082945926107 precision =  0.20203082945926107 total =  8257


In [76]:
for thresh in [1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A4, predict_test(A4,thresh,n=5,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.01 :  accuracy =  0.14328358208955225 precision =  0.14328358208955225 total =  5856
0.001 :  accuracy =  0.14382187423538048 precision =  0.14382187423538048 total =  5878
0.0001 :  accuracy =  0.14729630535845364 precision =  0.14729630535845364 total =  6020
1e-05 :  accuracy =  0.1605823342304869 precision =  0.1605823342304869 total =  6563
1e-06 :  accuracy =  0.185294837288965 precision =  0.185294837288965 total =  7573
1e-07 :  accuracy =  0.20594568142892097 precision =  0.20594568142892097 total =  8417
1e-08 :  accuracy =  0.20594568142892097 precision =  0.20594568142892097 total =  8417


In [ ]:
for thresh in [1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A4, predict_test(A4,thresh,n=8,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

# Best precision

# increasing

In [79]:
for thresh in [1.0e-1, 1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A0, predict_test(A0,thresh,n=1,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.1 :  accuracy =  0.0 precision =  0.0 total =  0
0.01 :  accuracy =  0.0 precision =  0.0 total =  0
0.001 :  accuracy =  0.0007333088897036765 precision =  0.0007339939278684149 total =  44
0.0001 :  accuracy =  0.03491550281657278 precision =  0.034948119974643845 total =  2095
1e-05 :  accuracy =  0.049581680610646314 precision =  0.04962799853201214 total =  2975
1e-06 :  accuracy =  0.058981367287757075 precision =  0.05903646615287092 total =  3539
1e-07 :  accuracy =  0.0724309189693677 precision =  0.0724985820571848 total =  4346
1e-08 :  accuracy =  0.0724309189693677 precision =  0.0724985820571848 total =  4346


In [80]:
for thresh in [1.0e-1, 1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A0, predict_test(A0,thresh,n=2,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.1 :  accuracy =  0.0 precision =  0.0 total =  0
0.01 :  accuracy =  0.0 precision =  0.0 total =  0
0.001 :  accuracy =  0.0001499950001666611 precision =  0.0001501351216094485 total =  9
0.0001 :  accuracy =  0.019966001133295556 precision =  0.01998465285423548 total =  1198
1e-05 :  accuracy =  0.04214859504683177 precision =  0.04218796917225503 total =  2529
1e-06 :  accuracy =  0.06346455118162728 precision =  0.06352383812097555 total =  3808
1e-07 :  accuracy =  0.08388053731542282 precision =  0.08395889634003936 total =  5033
1e-08 :  accuracy =  0.08388053731542282 precision =  0.08395889634003936 total =  5033


In [81]:
for thresh in [1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A0, predict_test(A0,thresh,n=3,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.01 :  accuracy =  0.0 precision =  0.0 total =  0
0.001 :  accuracy =  6.666444451851606e-05 precision =  6.672672071531044e-05 total =  4
0.0001 :  accuracy =  0.009716342788573714 precision =  0.009725419544256498 total =  583
1e-05 :  accuracy =  0.03286557114762841 precision =  0.03289627331264805 total =  1972
1e-06 :  accuracy =  0.057981400619979334 precision =  0.05803556534214126 total =  3479
1e-07 :  accuracy =  0.0823139228692377 precision =  0.08239081840322958 total =  4939
1e-08 :  accuracy =  0.0823139228692377 precision =  0.08239081840322958 total =  4939


In [82]:
for thresh in [1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A0, predict_test(A0,thresh,n=4,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.01 :  accuracy =  0.0 precision =  0.0 total =  0
0.001 :  accuracy =  0.0 precision =  0.0 total =  0
0.0001 :  accuracy =  0.005683143895203493 precision =  0.005688452940980216 total =  341
1e-05 :  accuracy =  0.022915902803239893 precision =  0.022937310245887965 total =  1375
1e-06 :  accuracy =  0.048131728942368585 precision =  0.048176692356454144 total =  2888
1e-07 :  accuracy =  0.0742808573047565 precision =  0.07435024855703466 total =  4457
1e-08 :  accuracy =  0.0742808573047565 precision =  0.07435024855703466 total =  4457


In [83]:
for thresh in [1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A0, predict_test(A0,thresh,n=5,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.01 :  accuracy =  0.0 precision =  0.0 total =  0
0.001 :  accuracy =  0.0 precision =  0.0 total =  0
0.0001 :  accuracy =  0.0034998833372220928 precision =  0.0035031528375537983 total =  210
1e-05 :  accuracy =  0.017149428352388255 precision =  0.017165448904013614 total =  1029
1e-06 :  accuracy =  0.040015332822239257 precision =  0.040052714109365097 total =  2401
1e-07 :  accuracy =  0.06683110562981234 precision =  0.06689353751709873 total =  4010
1e-08 :  accuracy =  0.06683110562981234 precision =  0.06689353751709873 total =  4010


# non-decreasing

In [ ]:
for thresh in [1.0e-1, 1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A1, predict_test(A1,thresh,n=1,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.1 :  accuracy =  0.003278914376248536 precision =  0.003281446042272746 total =  238
0.01 :  accuracy =  0.003278914376248536 precision =  0.003281446042272746 total =  238
0.001 :  accuracy =  0.00362333815526624 precision =  0.0036261357525955136 total =  263
0.0001 :  accuracy =  0.033409106564717225 precision =  0.03343490190130844 total =  2425
1e-05 :  accuracy =  0.04681408004408624 precision =  0.04685022542707055 total =  3398
1e-06 :  accuracy =  0.05627884549149273 precision =  0.0563222986667402 total =  4085
1e-07 :  accuracy =  0.06860921678032651 precision =  0.06866219029629528 total =  4980
1e-08 :  accuracy =  0.06860921678032651 precision =  0.06866219029629528 total =  4980


In [14]:
for thresh in [1.0e-1, 1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A1, predict_test(A1,thresh,n=2,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.1 :  accuracy =  0.003278914376248536 precision =  0.003281446042272746 total =  238
0.01 :  accuracy =  0.003278914376248536 precision =  0.003281446042272746 total =  238
0.001 :  accuracy =  0.0041055314458910246 precision =  0.0041087013470473875 total =  298
0.0001 :  accuracy =  0.023627471240614453 precision =  0.023645714128141846 total =  1715
1e-05 :  accuracy =  0.04491286078390852 precision =  0.04494753822608887 total =  3260
1e-06 :  accuracy =  0.06555073362264931 precision =  0.0656013456686291 total =  4758
1e-07 :  accuracy =  0.08438382585933733 precision =  0.08444897902907803 total =  6125
1e-08 :  accuracy =  0.08438382585933733 precision =  0.08444897902907803 total =  6125


In [15]:
for thresh in [1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A1, predict_test(A1,thresh,n=3,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.01 :  accuracy =  0.003278914376248536 precision =  0.003281446042272746 total =  238
0.001 :  accuracy =  0.0038024385203554454 precision =  0.0038053744019633526 total =  276
0.0001 :  accuracy =  0.01410759798856513 precision =  0.014118490534820555 total =  1024
1e-05 :  accuracy =  0.03648136667355514 precision =  0.036509534117387527 total =  2648
1e-06 :  accuracy =  0.06243714266032927 precision =  0.06248535068731128 total =  4532
1e-07 :  accuracy =  0.08465936488255149 precision =  0.08472473079733624 total =  6145
1e-08 :  accuracy =  0.08465936488255149 precision =  0.08472473079733624 total =  6145


In [16]:
for thresh in [1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A1, predict_test(A1,thresh,n=4,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.01 :  accuracy =  0.003278914376248536 precision =  0.003281446042272746 total =  238
0.001 :  accuracy =  0.003568230350623407 precision =  0.0035709853989438706 total =  259
0.0001 :  accuracy =  0.010277605565888269 precision =  0.01028554095603138 total =  746
1e-05 :  accuracy =  0.02792588000275539 precision =  0.027947441712969985 total =  2027
1e-06 :  accuracy =  0.05288971550595853 precision =  0.05293055191716417 total =  3839
1e-07 :  accuracy =  0.07686161052559069 precision =  0.07692095575562878 total =  5579
1e-08 :  accuracy =  0.07686161052559069 precision =  0.07692095575562878 total =  5579


In [17]:
for thresh in [1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A1, predict_test(A1,thresh,n=5,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.01 :  accuracy =  0.003278914376248536 precision =  0.003281446042272746 total =  238
0.001 :  accuracy =  0.0035406764483019906 precision =  0.0035434102221180495 total =  257
0.0001 :  accuracy =  0.0076599848453537235 precision =  0.007665899157578348 total =  556
1e-05 :  accuracy =  0.021822690638561685 precision =  0.021839540046050546 total =  1584
1e-06 :  accuracy =  0.045587931390783216 precision =  0.0456231300583215 total =  3309
1e-07 :  accuracy =  0.07041399738237927 precision =  0.07046836437838658 total =  5111
1e-08 :  accuracy =  0.07041399738237927 precision =  0.07046836437838658 total =  5111


# decreasing

In [18]:
for thresh in [1.0e-1, 1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A2, predict_test(A2,thresh,n=1,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.1 :  accuracy =  0.0 precision =  0.0 total =  0
0.01 :  accuracy =  0.0 precision =  0.0 total =  0
0.001 :  accuracy =  0.07352941176470588 precision =  0.09259259259259259 total =  20
0.0001 :  accuracy =  0.11029411764705882 precision =  0.1388888888888889 total =  30
1e-05 :  accuracy =  0.11029411764705882 precision =  0.1388888888888889 total =  30
1e-06 :  accuracy =  0.11029411764705882 precision =  0.1388888888888889 total =  30
1e-07 :  accuracy =  0.11029411764705882 precision =  0.1388888888888889 total =  30
1e-08 :  accuracy =  0.11029411764705882 precision =  0.1388888888888889 total =  30


In [19]:
for thresh in [1.0e-1, 1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A2, predict_test(A2,thresh,n=2,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.1 :  accuracy =  0.0 precision =  0.0 total =  0
0.01 :  accuracy =  0.0 precision =  0.0 total =  0
0.001 :  accuracy =  0.06985294117647059 precision =  0.08796296296296297 total =  19
0.0001 :  accuracy =  0.08455882352941177 precision =  0.10648148148148148 total =  23
1e-05 :  accuracy =  0.08455882352941177 precision =  0.10648148148148148 total =  23
1e-06 :  accuracy =  0.08455882352941177 precision =  0.10648148148148148 total =  23
1e-07 :  accuracy =  0.08455882352941177 precision =  0.10648148148148148 total =  23
1e-08 :  accuracy =  0.08455882352941177 precision =  0.10648148148148148 total =  23


In [20]:
for thresh in [1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A2, predict_test(A2,thresh,n=3,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.01 :  accuracy =  0.0 precision =  0.0 total =  0
0.001 :  accuracy =  0.07352941176470588 precision =  0.09259259259259259 total =  20
0.0001 :  accuracy =  0.08088235294117647 precision =  0.10185185185185185 total =  22
1e-05 :  accuracy =  0.08088235294117647 precision =  0.10185185185185185 total =  22
1e-06 :  accuracy =  0.08088235294117647 precision =  0.10185185185185185 total =  22
1e-07 :  accuracy =  0.08088235294117647 precision =  0.10185185185185185 total =  22
1e-08 :  accuracy =  0.08088235294117647 precision =  0.10185185185185185 total =  22


In [21]:
for thresh in [1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A2, predict_test(A2,thresh,n=4,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.01 :  accuracy =  0.0 precision =  0.0 total =  0
0.001 :  accuracy =  0.07352941176470588 precision =  0.09259259259259259 total =  20
0.0001 :  accuracy =  0.08088235294117647 precision =  0.10185185185185185 total =  22
1e-05 :  accuracy =  0.08088235294117647 precision =  0.10185185185185185 total =  22
1e-06 :  accuracy =  0.08088235294117647 precision =  0.10185185185185185 total =  22
1e-07 :  accuracy =  0.08088235294117647 precision =  0.10185185185185185 total =  22
1e-08 :  accuracy =  0.08088235294117647 precision =  0.10185185185185185 total =  22


In [22]:
for thresh in [1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A2, predict_test(A2,thresh,n=5,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.01 :  accuracy =  0.0 precision =  0.0 total =  0
0.001 :  accuracy =  0.07352941176470588 precision =  0.09259259259259259 total =  20
0.0001 :  accuracy =  0.08088235294117647 precision =  0.10185185185185185 total =  22
1e-05 :  accuracy =  0.08088235294117647 precision =  0.10185185185185185 total =  22
1e-06 :  accuracy =  0.08088235294117647 precision =  0.10185185185185185 total =  22
1e-07 :  accuracy =  0.08088235294117647 precision =  0.10185185185185185 total =  22
1e-08 :  accuracy =  0.08088235294117647 precision =  0.10185185185185185 total =  22


# non-increasing

In [23]:
for thresh in [1.0e-1, 1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A3, predict_test(A3,thresh,n=1,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.1 :  accuracy =  0.23931623931623933 precision =  0.27184466019417475 total =  112
0.01 :  accuracy =  0.3034188034188034 precision =  0.3446601941747573 total =  142
0.001 :  accuracy =  0.31837606837606836 precision =  0.3616504854368932 total =  149
0.0001 :  accuracy =  0.36324786324786323 precision =  0.41262135922330095 total =  170
1e-05 :  accuracy =  0.3803418803418803 precision =  0.4320388349514563 total =  178
1e-06 :  accuracy =  0.3803418803418803 precision =  0.4320388349514563 total =  178
1e-07 :  accuracy =  0.3803418803418803 precision =  0.4320388349514563 total =  178
1e-08 :  accuracy =  0.3803418803418803 precision =  0.4320388349514563 total =  178


In [24]:
for thresh in [1.0e-1, 1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A3, predict_test(A3,thresh,n=2,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.1 :  accuracy =  0.23504273504273504 precision =  0.2669902912621359 total =  110
0.01 :  accuracy =  0.297008547008547 precision =  0.33737864077669905 total =  139
0.001 :  accuracy =  0.3141025641025641 precision =  0.3567961165048544 total =  147
0.0001 :  accuracy =  0.358974358974359 precision =  0.4077669902912621 total =  168
1e-05 :  accuracy =  0.36752136752136755 precision =  0.4174757281553398 total =  172
1e-06 :  accuracy =  0.36752136752136755 precision =  0.4174757281553398 total =  172
1e-07 :  accuracy =  0.36752136752136755 precision =  0.4174757281553398 total =  172
1e-08 :  accuracy =  0.36752136752136755 precision =  0.4174757281553398 total =  172


In [25]:
for thresh in [1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A3, predict_test(A3,thresh,n=3,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.01 :  accuracy =  0.297008547008547 precision =  0.33737864077669905 total =  139
0.001 :  accuracy =  0.31196581196581197 precision =  0.35436893203883496 total =  146
0.0001 :  accuracy =  0.3504273504273504 precision =  0.39805825242718446 total =  164
1e-05 :  accuracy =  0.3525641025641026 precision =  0.40048543689320387 total =  165
1e-06 :  accuracy =  0.3525641025641026 precision =  0.40048543689320387 total =  165
1e-07 :  accuracy =  0.3525641025641026 precision =  0.40048543689320387 total =  165
1e-08 :  accuracy =  0.3525641025641026 precision =  0.40048543689320387 total =  165


In [26]:
for thresh in [1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A3, predict_test(A3,thresh,n=4,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.01 :  accuracy =  0.29273504273504275 precision =  0.3325242718446602 total =  137
0.001 :  accuracy =  0.31196581196581197 precision =  0.35436893203883496 total =  146
0.0001 :  accuracy =  0.3504273504273504 precision =  0.39805825242718446 total =  164
1e-05 :  accuracy =  0.3525641025641026 precision =  0.40048543689320387 total =  165
1e-06 :  accuracy =  0.3525641025641026 precision =  0.40048543689320387 total =  165
1e-07 :  accuracy =  0.3525641025641026 precision =  0.40048543689320387 total =  165
1e-08 :  accuracy =  0.3525641025641026 precision =  0.40048543689320387 total =  165


In [27]:
for thresh in [1.0e-2,1.0e-3,1.0e-4,1.0e-5,1.0e-6,1.0e-7, 1.0e-8]:
    total_right, accuracy, precision = test(A3, predict_test(A3,thresh,n=5,fallback=True))
    print(thresh, ": ", "accuracy = ", accuracy, "precision = ", precision, "total = ", total_right)

0.01 :  accuracy =  0.2905982905982906 precision =  0.3300970873786408 total =  136
0.001 :  accuracy =  0.3141025641025641 precision =  0.3567961165048544 total =  147
0.0001 :  accuracy =  0.3504273504273504 precision =  0.39805825242718446 total =  164
1e-05 :  accuracy =  0.3525641025641026 precision =  0.40048543689320387 total =  165
1e-06 :  accuracy =  0.3525641025641026 precision =  0.40048543689320387 total =  165
1e-07 :  accuracy =  0.3525641025641026 precision =  0.40048543689320387 total =  165
1e-08 :  accuracy =  0.3525641025641026 precision =  0.40048543689320387 total =  165
